In [1]:
# 단어-라벨 dict 불러오기
# onnx 불러오기

# 문장 입력하고
# 문장 tokenize and labeling (전처리)
# 문장을 신경망 안에 넣어서
# 결과값 해석

In [2]:
# 모듈 import
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))
import custom

import numpy as np
import pickle
import onnxruntime

from kiwipiepy import Kiwi
kiwi = Kiwi()

In [3]:
# 감정 list
targets = ['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오']

# 단어-라벨 dict 불러오기
with open("korean_word.pkl", mode = "rb") as f:
    word_dict = pickle.load(f)

print(len(word_dict))

30187


In [4]:
# 신경망 불러오기

F = onnxruntime.InferenceSession("encoder.onnx", providers=["CPUExecutionProvider"])


In [5]:
# 함수들 구현
def preprocess(text) :
    word_list = []
    text = custom.text_preprocess_kor(text, end_mark=True, chosung=False)
    temp = kiwi.tokenize(text, z_coda=True, compatible_jamo=True, split_complex=True)
    for w in temp :
        word_list.append(w.form)
    label = custom.word_vectorize(word_list, word_dict)
    label = [label] # 차원만 2차원으로 하나 늘려주기

    unk_list = custom.get_unk_words(word_list, word_dict) # 단어 사전에 없는단어 찾는 함수

    return label, unk_list

def cal(x) :
    x = np.array(x).astype(np.int64)
    inputs = {F.get_inputs()[0].name : x}
    outs = F.run(None, inputs)
    y = outs[0]
    return y

def get_argmax(y) :
    argmax = np.argmax(y, axis = -1).item()
    return f"감정 : {targets[argmax]}"

def get_softmax(y) :
    softmax = (np.exp(y - y.max())) / (np.sum(np.exp(y - y.max())))
    softmax = (softmax * 100).astype(np.int16).squeeze()
    text = ""
    for i in range(len(targets)) :
        temp = f"\t{targets[i]} : {softmax[i].item()}\n"
        text += temp
    return text
        

In [8]:
# 문장 입력하고
query = input("문장을 입력하세요 : ")
# 문장 tokenize and labeling (전처리)
label, unk = preprocess(query)
# 문장을 신경망 안에 넣어서
y = cal(label)
# 결과값 해석
print(y)
print(get_argmax(y))
print(get_softmax(y))

문장을 입력하세요 :  화났어


[[-2.276331   -0.52149147  0.9693935  -1.258428    1.2194386  -2.2651768
  -2.056033  ]]
감정 : 중립
	공포 : 1
	놀람 : 8
	분노 : 36
	슬픔 : 3
	중립 : 46
	행복 : 1
	혐오 : 1



In [7]:
# test = "안녕하세요 반갑습니다!"

# temp = kiwi.tokenize(test)
# print(temp)